# Problem 1

## libraries imports

In [ ]:
!pip install torch==1.9.0+cu102 torchvision==0.10.0+cu102 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 831.4 MB 2.9 kB/s 
     |████████████████████████████████| 22.1 MB 1.2 MB/s 
     |████████████████████████████████| 1.9 MB 23.1 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.1+cu113
    Uninstalling torchvision-0.13.1+cu113:
      Successfully uninstalled torchvision-0.13.1+cu113
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.12.1+cu113
    Uninstalling torchaudio-0.12.1+cu113:
      Successfully uninstalled torchaudio-0.12.1+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed.

In [1]:
import pandas as pd
import torch
import torch.nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

## Mnist dataset import

In [ ]:
from keras.datasets import mnist

In [2]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml ('mnist_784', cache = False )

In [3]:
X = mnist.data.astype('float32')
y = mnist.target.astype('int64')

## Dataset split into train,validation and test set

In [4]:
def data_split(X,y,train,validation,test):
  X_train = X[:int(len(X)*(round(train/100,2)))]
  y_train = y[:len(X_train)]
  X_validation = X[len(X_train):int(len(X)*(round((train+validation)/100,2)))]
  y_validation = y[len(X_train):len(X_validation)+len(X_train)]
  X_test = X[int(len(X)*(round((train+validation)/100,2))):]
  y_test = y[len(X_train) + len(X_validation):]

  return X_train.reset_index(drop=True),y_train.reset_index(drop=True),X_test.reset_index(drop=True),y_test.reset_index(drop=True),X_validation.reset_index(drop=True),y_validation.reset_index(drop=True)



In [5]:
 X_train,y_train,X_test,y_test,X_validation,y_validation = data_split(X,y,70,15,15)

## Dataset distribution check

Here we check if the distribution is 70-15-15  

In [ ]:
def distrib_check(y_train,y_test,y_val):
  y_df = y_train.to_frame()
  result = y_df.groupby(['class'])['class'].count() 
  
  y_df = y_test.to_frame()
  result2= y_df.groupby(['class'])['class'].count()

  y_df = y_val.to_frame()
  result3 = y_df.groupby(['class'])['class'].count()
  
  for i in range(0,10):
    dist_train = result[i]/(result[i]+result2[i]+ result3[i])
    dist_test = result2[i]/(result[i]+result2[i]+ result3[i])
    dist_val = result3[i]/(result[i]+result2[i]+ result3[i])
    print(f'Ratios : {i} , train : {round(dist_train*100,2)} % , test : {round(dist_test*100,2)} % , validation : {round(dist_val*100,2)} %'  )
distrib_check(y_train,y_test,y_validation)
    

Ratios : 0 , train : 69.9 % , test : 14.86 % , validation : 15.24 %
Ratios : 1 , train : 70.71 % , test : 15.09 % , validation : 14.19 %
Ratios : 2 , train : 69.63 % , test : 15.51 % , validation : 14.86 %
Ratios : 3 , train : 69.82 % , test : 14.82 % , validation : 15.36 %
Ratios : 4 , train : 69.94 % , test : 15.14 % , validation : 14.92 %
Ratios : 5 , train : 69.79 % , test : 14.83 % , validation : 15.38 %
Ratios : 6 , train : 70.61 % , test : 14.73 % , validation : 14.66 %
Ratios : 7 , train : 69.6 % , test : 14.85 % , validation : 15.55 %
Ratios : 8 , train : 69.61 % , test : 14.95 % , validation : 15.44 %
Ratios : 9 , train : 70.29 % , test : 15.21 % , validation : 14.5 %


So data is distributed properly throw all datasets

## Transform dataframe to tensors

In [6]:
X_train_tensor = torch.tensor(X_train.to_numpy())
X_test_tensor = torch.tensor(X_test.to_numpy())
X_validation_tensor = torch.tensor(X_validation.to_numpy())

y_train_torch = torch.tensor(y_train.to_numpy())
y_test_torch = torch.tensor(y_test.to_numpy())
y_validation_torch = torch.tensor(y_validation.to_numpy())

In [7]:
X_train_tensor = X_train_tensor/255.0
X_test_tensor = X_test_tensor/255.0
X_validation_tensor = X_validation_tensor/255.0

## Create dataset with only 2 and 3 digits

In [8]:
x_train_torch_2 = X_train_tensor[torch.where(y_train_torch == 2)[0]] 
x_train_torch_3 = X_train_tensor[torch.where(y_train_torch == 3)[0]] 

x_test_torch_2 = X_test_tensor[torch.where(y_test_torch == 2)[0]] 
x_test_torch_3 = X_test_tensor[torch.where(y_test_torch == 3)[0]] 


x_val_torch_2 = X_validation_tensor[torch.where(y_validation_torch == 2)[0]] 
x_val_torch_3 = X_validation_tensor[torch.where(y_validation_torch == 3)[0]] 

In [9]:
x_train_torch_23 = torch.cat((x_train_torch_2,x_train_torch_3),0)
x_test_torch_23 = torch.cat((x_test_torch_2,x_test_torch_3),0)
x_validation_torch_23 = torch.cat((x_val_torch_2,x_val_torch_3),0)

In [10]:
y_train_torch_2 = y_train_torch[torch.where(y_train_torch == 2 )]
y_train_torch_3 = y_train_torch[torch.where(y_train_torch == 3 )]

y_test_torch_2 = y_test_torch[torch.where(y_test_torch == 2 )]
y_test_torch_3 = y_test_torch[torch.where(y_test_torch == 3 )]

y_val_torch_2 = y_validation_torch[torch.where(y_validation_torch == 2 )]
y_val_torch_3 = y_validation_torch[torch.where(y_validation_torch == 3 )]



In [11]:
y_train_torch_23 = torch.cat((y_train_torch_2,y_train_torch_3),0)
y_test_torch_23 = torch.cat((y_test_torch_2,y_test_torch_3),0)
y_val_torch_23 = torch.cat((y_val_torch_2,y_val_torch_3),0)

In [ ]:
y_val_torch_23.size()

torch.Size([2136])

# Problem 2: **KNN** model 

## a) Here I made a KNN function. By default it will use Euclidean distance in order to compute distance between points 

In [ ]:
def KNN_train(X_train,y_train,X_test,k,distance='Euclidean'):
  device = torch.device('cuda:0')                               # Set the device in order to use the GPU                                       
  d = torch.tensor                                              # Set empty tensor for distance       
  if distance == 'Euclidean' :                                  # Conditions over distance that we will choose 
    d = EuclideanDistance(X_test,X_train)
  elif distance == 'Manhattan' : 
    d = MannhattanDistance(X_test,X_train)
  else :
    d = Minkowsky(X_test,X_train)

  dist, idx = d.topk(k=k, largest=False)                        # Index of top K distances 
  
  pred = KNN_prediction(dist,idx,y_train)                       # Function that returns predictions for each X_test
  return pred                                                   # Return prediction
  
def KNN_prediction(dist,idx,y_train):                           # Function that returns predictions based on index of the k-nearest distances
  predict_top_k = idx.apply_(lambda x : y_train[x])             # Retrieve labels of y_train by using indexes
  predictions = torch.mode(predict_top_k, dim = 1)[0]           
  return predictions 

def EuclideanDistance(X_test,X_train):                          # Euclidean Distance
  d = torch.cdist(X_test,X_train,p=2)
  return d

def MannhattanDistance(X_test,X_train):                         # MannhattanDistance
  d = torch.cdist(X_test,X_train,p=1)
  return d
  
def Minkowsky(X_test,X_train):
  d = torch.cdist(X_test,X_train,p=3)
  return d

Here we apply our KNN model to train the train set with k = 5 and we found an accuracy of 99.6% for the train set. But we can't be sufficient by this result. We have to test our algorithm to the validation set and see if the model doesn't overfit the train set.

In [ ]:
pred =KNN_train(x_train_torch_23,y_train_torch_23,x_train_torch_23,5)

In [ ]:
train_acc = torch.sum(pred == y_train_torch_23)
print("Train accuracy on 2 and 3 digit" , round(float(train_acc)/ pred.size()[0] *100,4),'%')

Train accuracy on 2 and 3 digit 99.5839 %


We apply KNN on validation set. with k = 5. We see an accuracy of 99.5%. So the model is pretty accurate. We can trust our knn algorithm with those parameters. 

In [ ]:
pred =KNN_train(x_train_torch_23,y_train_torch_23,x_validation_torch_23,5)

1


In [ ]:
val_acc = torch.sum(pred == y_val_torch_23)
print("Validation accuracy on 2 and 3 digit " , round(float(val_acc)/ pred.size()[0] *100,4),'%')


Validation accuracy on 2 and 3 digit  99.5318 %


Finally let's perform knn on our test set. We can see that the accuracy is equal to 99.6 %. We can assume that our model generalize well the situation. 

In [ ]:
pred =KNN_train(x_train_torch_23,y_train_torch_23,x_test_torch_23,5)

In [ ]:
test_acc = torch.sum(pred == y_test_torch_23)
print("Validation accuracy on 2 and 3 digit " , round(float(test_acc)/ pred.size()[0] *100,4),'%')


Validation accuracy on 2 and 3 digit  99.6265 %


## b) Hyperparameters we can tune are the number of k neighboors and the distance calculation. 
In other term we can thune our model to get the best k by testing mutliple k values.
And we can test different type of distance computation. We can test the manhattan distance or even the jackard distance.  


## c) First options for K : Let's try other k for the KNN 

In [ ]:
for k in range(1,20):
  pred =KNN_train(x_train_torch_23,y_train_torch_23,x_validation_torch_23,k)
  val_acc = torch.sum(pred == y_val_torch_23)
  print(f"k = {k} Validation accuracy on 2 and 3 digit " , round(float(val_acc)/ pred.size()[0] *100,4),'%')

k = 1 Validation accuracy on 2 and 3 digit  99.3914 %
k = 2 Validation accuracy on 2 and 3 digit  99.4382 %
k = 3 Validation accuracy on 2 and 3 digit  99.485 %
k = 4 Validation accuracy on 2 and 3 digit  99.5318 %
k = 5 Validation accuracy on 2 and 3 digit  99.5318 %
k = 6 Validation accuracy on 2 and 3 digit  99.5787 %
k = 7 Validation accuracy on 2 and 3 digit  99.6723 %
k = 8 Validation accuracy on 2 and 3 digit  99.6723 %
k = 9 Validation accuracy on 2 and 3 digit  99.6723 %
k = 10 Validation accuracy on 2 and 3 digit  99.6723 %
k = 11 Validation accuracy on 2 and 3 digit  99.7191 %
k = 12 Validation accuracy on 2 and 3 digit  99.7659 %
k = 13 Validation accuracy on 2 and 3 digit  99.7191 %
k = 14 Validation accuracy on 2 and 3 digit  99.7191 %
k = 15 Validation accuracy on 2 and 3 digit  99.7191 %
k = 16 Validation accuracy on 2 and 3 digit  99.7191 %
k = 17 Validation accuracy on 2 and 3 digit  99.6255 %
k = 18 Validation accuracy on 2 and 3 digit  99.6255 %
k = 19 Validation ac

So we can find that k = 12 is the optimum for the validation test. 
After k=12 the performance of the model decreases. 

Let's try k=12 on the test set. We found a good accuracy but lower than the accuracy for k = 5

In [ ]:
pred =KNN_train(x_train_torch_23,y_train_torch_23,x_test_torch_23,12)
test_acc = torch.sum(pred == y_test_torch_23)
print("Validation accuracy on 2 and 3 digit " , round(float(test_acc)/ pred.size()[0] *100,4),'%')

Validation accuracy on 2 and 3 digit  99.5798 %


Let's try k=6. We found an accuracy of 99.72% for the test set. 

In [ ]:
pred =KNN_train(x_train_torch_23,y_train_torch_23,x_test_torch_23,6)
test_acc = torch.sum(pred == y_test_torch_23)
print("Validation accuracy on 2 and 3 digit " , round(float(test_acc)/ pred.size()[0] *100,4),'%')



Validation accuracy on 2 and 3 digit  99.7199 %


Second option let's try an other distance computation for knn. I already implemented the distance inside the knn algorithm. We can now choose the distance we want to compute.

Let's try manhattan distance for k between 1 and 9 : The Manhattan Distance between two points (X1, Y1) and (X2, Y2) is given by |X1 – X2| + |Y1 – Y2|.

In [ ]:
for k in range(1,10):
  pred =KNN_train(x_train_torch_23,y_train_torch_23,x_validation_torch_23,k,'Manhattan')
  val_acc = torch.sum(pred == y_val_torch_23)
  print(f" k = {k} Validation accuracy on 2 and 3 digit " , round(float(val_acc)/ pred.size()[0] *100,4),'%')

 k = 1 Validation accuracy on 2 and 3 digit  99.3914 %
 k = 2 Validation accuracy on 2 and 3 digit  99.2509 %
 k = 3 Validation accuracy on 2 and 3 digit  99.3914 %
 k = 4 Validation accuracy on 2 and 3 digit  99.3446 %
 k = 5 Validation accuracy on 2 and 3 digit  99.2509 %
 k = 6 Validation accuracy on 2 and 3 digit  99.2978 %
 k = 7 Validation accuracy on 2 and 3 digit  99.485 %
 k = 8 Validation accuracy on 2 and 3 digit  99.485 %
 k = 9 Validation accuracy on 2 and 3 digit  99.5318 %


Let's try the minkowsky distance with power equal to 3. 
The Minkowski distance between two variabes X and Y is defined as
(∑i=1n|Xi−Yi|^p)^1/p

In [ ]:
pred =KNN_train(x_train_torch_23,y_train_torch_23,x_validation_torch_23,6,'Minkowsky')
val_acc = torch.sum(pred == y_val_torch_23)
print(f"Validation accuracy on 2 and 3 digit " , round(float(val_acc)/ pred.size()[0] *100,4),'%')

 k = 9 Validation accuracy on 2 and 3 digit  99.6723 %


## d) Reporting results

By testing different methods for each hyperparameters. **The best accuracy for the test set is : 99.72 % by using k = 6 and the Euclidean distance**

Resume of what we have tried :

1. Perform KNN with k = 5 and with euclidean distance. Test Accuracy = 99.53%
2. Iterating over each k = 1 to k = 19 with euclidean distance in order to find the optimum k for the validation test. Optimum k = 12, Best test Accuracy = 99.57%
3. **k = 6 best performance for test set. Test Accuracy = 99.72%**
4. Iteratting over k=1 to k=10 with Manhattan distance. Results are very good but lower than k = 6 problem
5. Minkowski power = 3 distance and k=6 . Test accuracy = 99.67%

# Problem 3 : KNN multiclass

We are gonna use the same code and perform it with all the labels values. Let's perfomr multi class KNN.

## a) Let's train our model, We have 98.01 accuracy with the train set

In [ ]:
pred =KNN_train(X_train_tensor,y_train_torch,X_train_tensor,5)

In [ ]:
train_acc = torch.sum(pred == y_train_torch)
print("Train accuracy" , round(float(train_acc)/ pred.size()[0] *100,4),'%')

Train accuracy 98.0184 %


We have 96.9% of accuracy for the validation set

In [ ]:
pred =KNN_train(X_train_tensor,y_train_torch,X_validation_tensor,5,'Euclidean')

In [ ]:
val_acc = torch.sum(pred == y_validation_torch)
print("Validation accuracy on multiclass KNN" , round(float(val_acc)/ pred.size()[0] *100,4),'%')

Validation accuracy on multiclass KNN 96.9429 %


Test accuracy on Multiclass KKN is 96.7%

In [ ]:
pred =KNN_train(X_train_tensor,y_train_torch,X_test_tensor,5)

In [ ]:
test_acc = torch.sum(pred == y_test_torch)
print("Test accuracy on multiclass KNN" , round(float(test_acc)/ pred.size()[0] *100,2),'%')

Test accuracy on multiclass KNN tensor(96.6762) %


## b) Hyperparameters that we can tune are stil the same, we can tune the value of k and distance calculation formula

in other term we can thune our model to get the best k by testing mutliple k values. And we can test different type of distance computation. We can test the manhattan distance or even the jackard distance.

## c) Let's tune k parameter and test different distance computation

What we can learn is that the accuracy of the model is decreasing slowly starting to k = 5. The optimum k for the validation set is k = 4 with an accuracy of 97.1%

In [ ]:
for k in range(1,21):
  pred =KNN_train(X_train_tensor,y_train_torch,X_validation_tensor,k,'Euclidean')
  val_acc = torch.sum(pred == y_validation_torch)
  print(f"k = {k} Validation accuracy " , round(float(val_acc)/ pred.size()[0] *100,4),'%')

k = 1 Validation accuracy  96.9143 %
k = 2 Validation accuracy  96.4476 %
k = 3 Validation accuracy  97.0571 %
k = 4 Validation accuracy  97.0762 %
k = 5 Validation accuracy  96.9429 %
k = 6 Validation accuracy  96.9143 %
k = 7 Validation accuracy  96.8857 %
k = 8 Validation accuracy  96.8762 %
k = 9 Validation accuracy  96.8571 %
k = 10 Validation accuracy  96.7714 %
k = 11 Validation accuracy  96.7238 %
k = 12 Validation accuracy  96.6 %
k = 13 Validation accuracy  96.5714 %
k = 14 Validation accuracy  96.381 %
k = 15 Validation accuracy  96.3714 %
k = 16 Validation accuracy  96.419 %
k = 17 Validation accuracy  96.3619 %
k = 18 Validation accuracy  96.2476 %
k = 19 Validation accuracy  96.2857 %
k = 20 Validation accuracy  96.2 %


Let's apply this optimum k to the test set and see if the accuracy is better. The performance is not that better.

In [ ]:
pred =KNN_train(X_train_tensor,y_train_torch,X_test_tensor,4)
test_acc = torch.sum(pred == y_test_torch)
print("Test accuracy on multiclass KNN" , round(float(test_acc)/ pred.size()[0] *100,2),'%')

Test accuracy on multiclass KNN 96.57 %


Let's try an other distance calculation. The performance is not better then previous one. 

In [ ]:
pred =KNN_train(X_train_tensor,y_train_torch,X_validation_tensor,5,'Manhattan')
val_acc = torch.sum(pred == y_validation_torch)
print("Validation accuracy on multiclass KNN" , round(float(val_acc)/ pred.size()[0] *100,4),'%')

Validation accuracy on multiclass KNN 96.3714 %


# Problem 4: Binary Classification via soft-margin SVM


First of all before starting coding. We have to replace labels values of digit 2 and 3 by 1 and -1 in order to perform our classification using Soft-margin SVM

In [12]:
y_train_torch_svm= torch.where(y_train_torch_23==2,1,-1)
y_test_torch_svm= torch.where(y_test_torch_23==2,1,-1)
y_val_torch_svm= torch.where(y_val_torch_23==2,1,-1)

The goal of this part is to prepare the mini batch gradient descent using mini batch. We will shuffle the training set

In [13]:
import torch
from torch.utils.data.dataset import TensorDataset
from torch.utils.data import DataLoader

train_data = TensorDataset(x_train_torch_23,y_train_torch_svm)  
test_data = TensorDataset(x_test_torch_23,y_test_torch_svm) 
val_data = TensorDataset(x_validation_torch_23,y_val_torch_svm) 

train_dataloader = DataLoader(train_data,batch_size = 64,shuffle = True )
test_dataloader = DataLoader(test_data,batch_size = 64, shuffle = False)
val_dataloader = DataLoader(val_data,batch_size = 64, shuffle = False)

## a) SVM Classifier

This is our SVM classifier. We are using a linear model with a bias value wx+b 

In [34]:
class SVM(torch.nn.Module):
  def __init__(self,nb_of_features):
    
    super(SVM,self).__init__()
    # We initialize our layer that will helps to create the decision boundary
    self.linear = torch.nn.Linear(nb_of_features, 1, bias=True) 

  def forward(self,x):
    y = self.linear(x)
    return y.squeeze()


# Initialization of our model
model1 = SVM(x_train_torch_23.shape[1])


Function to calculate the accuracy of the model

In [33]:
def accuracy(y_pred,y):
   return round(float(torch.sum(y_pred == y )/y.size()[0])*100,2)

## b) Train for 10 epochs with batch size 64 (1 epoch is equal to one entire passing of the
train set; i.e., the entire train set is used for training once).

Here we initialize hyperparameters of our model

In [35]:
#Set the model in train mode
model1.train()
C=1  # Not usefull now but we will use this condition later in this notebook
learning_rate = 0.001 # We set the learning rate 
epochs = 10          
total_loss = 0        #Loss of our algorithm
nb_of_batch = 0       #Count the number of batch
device = torch.device('cuda:0')

Here we train our model in order to have the best values for the weights of our model. Reminder : We try to maximize the distance between the neareast point of each and the margin. And we try to minimize the weights values.

In [36]:
#We initialize empty tensor for the outputs 

for epoch in range(epochs):
  output = torch.empty(0)                 
  y_train_values = torch.empty(0)
  for batch in train_dataloader: # We apply the mini batch here 
    x_train,y_train = batch      # Instanciate x and y regarding the batch (64)
    y_pred = model1.forward(x_train.float())    
    # Our loss function that use the hinge loss and the norm of the weights 
    loss = (0.5 * torch.norm(model1.linear.weight.squeeze())**2 )+ (C* torch.clamp(1-y_pred*y_train,min=0)).mean()
    loss.backward()
    total_loss += float(loss)  # We add the loss of this batch to the total loss
    nb_of_batch += 1
    

    # Just to retrieve values of y_train because the dataset is shuffleld and predictions
    y_train_values = torch.cat((y_train_values,y_train),0)
    output = torch.cat((output,y_pred),0)

    with torch.no_grad():  # Here we apply de gradient descned regarding the learning rate and the weights
      for param in model1.parameters():
        param -= learning_rate*param.grad
    model1.zero_grad()
  final_pred = torch.where(output >=0, 2, 3 )
  y_true = torch.where(y_train_values>= 0,2,3)
  print('Epochs : '+ str(epoch) + ' Loss = '+ str(round((total_loss/nb_of_batch),4)) + ' Accuracy = ' + str(accuracy(y_true,final_pred)))

# Final prediction we make the decision if we consider predictions are classe 2 or 3 if their higher than 0

Epochs : 0 Loss = 0.8333 Accuracy = 79.31
Epochs : 1 Loss = 0.6664 Accuracy = 92.98
Epochs : 2 Loss = 0.5865 Accuracy = 94.18
Epochs : 3 Loss = 0.5388 Accuracy = 94.51
Epochs : 4 Loss = 0.5065 Accuracy = 94.97
Epochs : 5 Loss = 0.4828 Accuracy = 95.14
Epochs : 6 Loss = 0.4647 Accuracy = 95.3
Epochs : 7 Loss = 0.4503 Accuracy = 95.39
Epochs : 8 Loss = 0.4386 Accuracy = 95.5
Epochs : 9 Loss = 0.4289 Accuracy = 95.47


We can see that our loss function decrease over epochs and the accuracy increases

The accuracy for our Train set is 95.5% 

In [29]:
print('Train accuracy',accuracy(y_true,final_pred))

Train accuracy 95.49


Let's try on on the validation set. We found a better accuracy ! Our model genelralize well this situation

## c) How should we choose the number of iterations to achieve good generalization? Train
until you think the model has achieved good generalization

As we seen in the previous example we should choose the number of iterations we the loss starts to be stable. If we do too many epochs we might be overtting the model

Let's try to see when the loss is stable for our model

In [41]:
#Set the model in train mode
model2 = SVM(x_train_torch_23.shape[1])

model2.train()
C=1  # Not usefull now but we will use this condition later in this notebook
learning_rate = 0.001 # We set the learning rate 
epochs = 40          
total_loss = 0        #Loss of our algorithm
nb_of_batch = 0       #Count the number of batch
device = torch.device('cuda:0')

#We initialize empty tensor for the outputs 

for epoch in range(epochs):
  output = torch.empty(0)                 
  y_train_values = torch.empty(0)
  for batch in train_dataloader: # We apply the mini batch here 
    x_train,y_train = batch      # Instanciate x and y regarding the batch (64)
    y_pred = model2.forward(x_train.float())    
    # Our loss function that use the hinge loss and the norm of the weights 
    loss = (0.5 * torch.norm(model2.linear.weight.squeeze())**2 )+ (C* torch.clamp(1-y_pred*y_train,min=0)).mean()
    loss.backward()
    total_loss += float(loss)  # We add the loss of this batch to the total loss
    nb_of_batch += 1
    

    # Just to retrieve values of y_train because the dataset is shuffleld and predictions
    y_train_values = torch.cat((y_train_values,y_train),0)
    output = torch.cat((output,y_pred),0)

    with torch.no_grad():  # Here we apply de gradient descned regarding the learning rate and the weights
      for param in model2.parameters():
        param -= learning_rate*param.grad
    model2.zero_grad()
  final_pred = torch.where(output >=0, 2, 3 )
  y_true = torch.where(y_train_values>= 0,2,3)
  print('Epochs : '+ str(epoch) + ' Loss = '+ str(round((total_loss/nb_of_batch),4)) + ' Accuracy = ' + str(accuracy(y_true,final_pred)))

Epochs : 0 Loss = 0.8209 Accuracy = 85.94
Epochs : 1 Loss = 0.6574 Accuracy = 93.84
Epochs : 2 Loss = 0.5801 Accuracy = 94.54
Epochs : 3 Loss = 0.534 Accuracy = 94.94
Epochs : 4 Loss = 0.5027 Accuracy = 95.14
Epochs : 5 Loss = 0.4797 Accuracy = 95.28
Epochs : 6 Loss = 0.4621 Accuracy = 95.41
Epochs : 7 Loss = 0.4481 Accuracy = 95.45
Epochs : 8 Loss = 0.4367 Accuracy = 95.39
Epochs : 9 Loss = 0.4272 Accuracy = 95.62
Epochs : 10 Loss = 0.4193 Accuracy = 95.56
Epochs : 11 Loss = 0.4125 Accuracy = 95.63
Epochs : 12 Loss = 0.4067 Accuracy = 95.67
Epochs : 13 Loss = 0.4017 Accuracy = 95.69
Epochs : 14 Loss = 0.3972 Accuracy = 95.63
Epochs : 15 Loss = 0.3933 Accuracy = 95.68
Epochs : 16 Loss = 0.3899 Accuracy = 95.69
Epochs : 17 Loss = 0.3868 Accuracy = 95.75
Epochs : 18 Loss = 0.384 Accuracy = 95.66
Epochs : 19 Loss = 0.3815 Accuracy = 95.66
Epochs : 20 Loss = 0.3792 Accuracy = 95.68
Epochs : 21 Loss = 0.3771 Accuracy = 95.75
Epochs : 22 Loss = 0.3753 Accuracy = 95.72
Epochs : 23 Loss = 0.37

We can see that regarding the results we see that the model start to be "stable" starting the 10th epoch.

Let's perform the first model on the validation and tests set

In [30]:
nb_of_batch = 0
loss = 0 
total_loss=0
model1.eval() # Model in evaluation mode
output = torch.empty(0)
for batch in test_dataloader: 
    x_test,y_test = batch
    y_pred = model1.forward(x_test)
    loss += (0.5 * torch.norm(model1.linear.weight.squeeze())**2 )+ (C* torch.clamp(1-y_pred*y_test,min=0)).mean()
    total_loss += float(loss)
    nb_of_batch += 1
    output = torch.cat((output,y_pred),0)
final_pred = torch.where(output >=0, 2, 3 )

print("Loss : " + str(total_loss/nb_of_batch) + " Accuracy = " + str(float(accuracy(final_pred,y_test_torch_23))))


Loss : 6.030509240487042 Accuracy = 96.64


On the test set the accuracy is up to 96.6% Actually the loss can be even lower. But the accuracy is pretty good

In [31]:
nb_of_batch = 0
loss = 0 
total_loss=0
model1.eval()
output = torch.empty(0)

for batch in val_dataloader: 
    x_val,y_val = batch
    y_pred = model1.forward(x_val)
    loss += (0.5 * torch.norm(model1.linear.weight.squeeze())**2 )+ (C* torch.clamp(1-y_pred*y_val,min=0)).mean()
    total_loss += float(loss)
    nb_of_batch += 1
    output = torch.cat((output,y_pred),0)
final_pred = torch.where(output >=0, 2, 3 )

print("Loss : " + str(total_loss/nb_of_batch) + " Accuracy = " + str(float(accuracy(final_pred,y_val_torch_23))))

Loss : 5.73075587609235 Accuracy = 96.4


##  d) What are the hyperparameters you can tune? 


We can tune the learning rate, the value of C, the number of epochs, the value of gamma in the loss function.

C parameter adds a penalty for each misclassified data point. If c is small, the penalty for misclassified points is low so a decision boundary with a large margin is chosen at the expense of a greater number of misclassifications

For high values of gamma, the points need to be very close to each other in order to be considered in the same class. Therefore, models with very large gamma values tend to overfit.




##e) Try at least two other options for each hyperparameter. Report the performance for
each optio
**Let's try this rule :**

**0.0001 < gamma < 10**

**0.1 < c < 100**

We are gonna try every value

In [76]:
#Set the model in train mode
model2 = SVM(x_train_torch_23.shape[1])

model2.train()
C=[0.1,1,10,100]  # Not usefull now but we will use this condition later in this notebook
learning_rate = 0.001 # We set the learning rate 
epochs = 10
gamma = [0.001,0.01,0.1,1,10,10]          
total_loss = 0        #Loss of our algorithm
nb_of_batch = 0       #Count the number of batch
device = torch.device('cuda:0')

#We initialize empty tensor for the outputs 
for C_val in C:
  for gamma_val in gamma:
    for epoch in range(epochs):
      output = torch.empty(0)                 
      y_train_values = torch.empty(0)
      for batch in train_dataloader: # We apply the mini batch here 
        x_train,y_train = batch      # Instanciate x and y regarding the batch (64)
        y_pred = model2.forward(x_train.float())    
        # Our loss function that use the hinge loss and the norm of the weights 
        loss = ((0.5/gamma_val) * torch.norm(model2.linear.weight.squeeze())**2 )+ (C_val* torch.clamp(1-y_pred*y_train,min=0)).mean()
        loss.backward()
        total_loss += float(loss)  # We add the loss of this batch to the total loss
        nb_of_batch += 1
        

        # Just to retrieve values of y_train because the dataset is shuffleld and predictions
        y_train_values = torch.cat((y_train_values,y_train),0)
        output = torch.cat((output,y_pred),0)

        with torch.no_grad():  # Here we apply de gradient descned regarding the learning rate and the weights
          for param in model2.parameters():
            param -= learning_rate*param.grad
        model2.zero_grad()
      final_pred = torch.where(output >=0, 2, 3 )
      y_true = torch.where(y_train_values>= 0,2,3)
      print('C : ' + str(C_val) + 'Gamma :' + str(gamma_val) + 'Epochs : '+ str(epoch) + ' Loss = '+ str(round((total_loss/nb_of_batch),4)) + ' Accuracy = '+ str(accuracy(y_true,final_pred)))

C : 0.1Gamma :0.001Epochs : 0 Loss = 1.21 Accuracy = 49.31
C : 0.1Gamma :0.001Epochs : 1 Loss = 0.655 Accuracy = 49.4
C : 0.1Gamma :0.001Epochs : 2 Loss = 0.47 Accuracy = 49.4
C : 0.1Gamma :0.001Epochs : 3 Loss = 0.3775 Accuracy = 49.4
C : 0.1Gamma :0.001Epochs : 4 Loss = 0.322 Accuracy = 49.4
C : 0.1Gamma :0.001Epochs : 5 Loss = 0.285 Accuracy = 49.4
C : 0.1Gamma :0.001Epochs : 6 Loss = 0.2586 Accuracy = 49.4
C : 0.1Gamma :0.001Epochs : 7 Loss = 0.2388 Accuracy = 49.4
C : 0.1Gamma :0.001Epochs : 8 Loss = 0.2234 Accuracy = 49.4
C : 0.1Gamma :0.001Epochs : 9 Loss = 0.211 Accuracy = 49.4
C : 0.1Gamma :0.01Epochs : 0 Loss = 0.2009 Accuracy = 49.4
C : 0.1Gamma :0.01Epochs : 1 Loss = 0.1925 Accuracy = 49.4
C : 0.1Gamma :0.01Epochs : 2 Loss = 0.1854 Accuracy = 49.4
C : 0.1Gamma :0.01Epochs : 3 Loss = 0.1792 Accuracy = 49.4
C : 0.1Gamma :0.01Epochs : 4 Loss = 0.1739 Accuracy = 49.4
C : 0.1Gamma :0.01Epochs : 5 Loss = 0.1693 Accuracy = 49.4
C : 0.1Gamma :0.01Epochs : 6 Loss = 0.1652 Accuracy =

The beter result is the last iteration when we got an accuracy of 97.70 %  Let's try it with the validation set

On the validation test we have an accuracy of 98.03 % the best on for the SVM until now. 

In [77]:
nb_of_batch = 0
loss = 0 
total_loss=0
gamma = 10
C = 100
model2.eval() # Model in evaluation mode
output = torch.empty(0)
for batch in test_dataloader: 
    x_test,y_test = batch
    y_pred = model2.forward(x_test)
    loss += ((0.5/gamma) * torch.norm(model2.linear.weight.squeeze())**2 )+ (C* torch.clamp(1-y_pred*y_test,min=0)).mean()
    total_loss += float(loss)
    nb_of_batch += 1
    output = torch.cat((output,y_pred),0)
final_pred = torch.where(output >=0, 2, 3 )

print("Loss : " + str(total_loss/nb_of_batch) + " Accuracy = " + str(float(accuracy(final_pred,y_test_torch_23))))

Loss : 113.43609630360322 Accuracy = 98.04


Let's perform this model on the test set

In [79]:
nb_of_batch = 0
loss = 0 
total_loss=0
C= 100
gamma = 10
model2.eval()
output = torch.empty(0)

for batch in val_dataloader: 
    x_val,y_val = batch
    y_pred = model2.forward(x_val)
    loss += ((0.5/gamma) * torch.norm(model2.linear.weight.squeeze())**2 )+ (C* torch.clamp(1-y_pred*y_val,min=0)).mean()
    total_loss += float(loss)
    nb_of_batch += 1
    output = torch.cat((output,y_pred),0)
final_pred = torch.where(output >=0, 2, 3 )

print("Loss : " + str(total_loss/nb_of_batch) + " Accuracy = " + str(float(accuracy(final_pred,y_val_torch_23))))

Loss : 126.39813106200275 Accuracy = 97.47


## f) You can try more options if you want. What is the final test accuracy

Let's try to find the better learning rate

In [80]:
model2 = SVM(x_train_torch_23.shape[1])

model2.train()
C=100  # Not usefull now but we will use this condition later in this notebook
learning_rate = [0.0001,0.001,0.002,0.005,0.0075,0.01,0.1] # We set the learning rate 
epochs = 10
gamma =10       
total_loss = 0        #Loss of our algorithm
nb_of_batch = 0       #Count the number of batch
device = torch.device('cuda:0')
for lr in learning_rate:
  for epoch in range(epochs):
      output = torch.empty(0)                 
      y_train_values = torch.empty(0)
      for batch in train_dataloader: # We apply the mini batch here 
        x_train,y_train = batch      # Instanciate x and y regarding the batch (64)
        y_pred = model2.forward(x_train.float())    
        # Our loss function that use the hinge loss and the norm of the weights 
        loss = ((0.5/gamma_val) * torch.norm(model2.linear.weight.squeeze())**2 )+ (C_val* torch.clamp(1-y_pred*y_train,min=0)).mean()
        loss.backward()
        total_loss += float(loss)  # We add the loss of this batch to the total loss
        nb_of_batch += 1
          

        # Just to retrieve values of y_train because the dataset is shuffleld and predictions
        y_train_values = torch.cat((y_train_values,y_train),0)
        output = torch.cat((output,y_pred),0)

        with torch.no_grad():  # Here we apply de gradient descned regarding the learning rate and the weights
            for param in model2.parameters():
              param -= lr*param.grad
            model2.zero_grad()
      final_pred = torch.where(output >=0, 2, 3 )
      y_true = torch.where(y_train_values>= 0,2,3)
      print('Learning rate : ' + str(lr) + 'Epochs : '+ str(epoch) + ' Loss = '+ str(round((total_loss/nb_of_batch),4)) + ' Accuracy = '+ str(accuracy(y_true,final_pred)))

Learning rate : 0.0001Epochs : 0 Loss = 23.2929 Accuracy = 93.27
Learning rate : 0.0001Epochs : 1 Loss = 17.9726 Accuracy = 96.21
Learning rate : 0.0001Epochs : 2 Loss = 15.6559 Accuracy = 96.5
Learning rate : 0.0001Epochs : 3 Loss = 14.2982 Accuracy = 96.73
Learning rate : 0.0001Epochs : 4 Loss = 13.3877 Accuracy = 96.85
Learning rate : 0.0001Epochs : 5 Loss = 12.7239 Accuracy = 96.9
Learning rate : 0.0001Epochs : 6 Loss = 12.2103 Accuracy = 96.98
Learning rate : 0.0001Epochs : 7 Loss = 11.8008 Accuracy = 97.0
Learning rate : 0.0001Epochs : 8 Loss = 11.4634 Accuracy = 97.03
Learning rate : 0.0001Epochs : 9 Loss = 11.1778 Accuracy = 97.11
Learning rate : 0.001Epochs : 0 Loss = 10.9404 Accuracy = 96.93
Learning rate : 0.001Epochs : 1 Loss = 10.7048 Accuracy = 97.18
Learning rate : 0.001Epochs : 2 Loss = 10.48 Accuracy = 97.21
Learning rate : 0.001Epochs : 3 Loss = 10.2721 Accuracy = 97.34
Learning rate : 0.001Epochs : 4 Loss = 10.0875 Accuracy = 97.37
Learning rate : 0.001Epochs : 5 Los

Loss : 849.0836720186121 Accuracy = 97.11


Learning rate equal to 0.002 is the optimum 

In [81]:
model2 = SVM(x_train_torch_23.shape[1])

model2.train()
C=100
gamma = 100  # Not usefull now but we will use this condition later in this notebook
learning_rate = 0.001 # We set the learning rate 
epochs = 10
total_loss = 0        #Loss of our algorithm
nb_of_batch = 0       #Count the number of batch
device = torch.device('cuda:0')


for epoch in range(epochs):
  output = torch.empty(0)                 
  y_train_values = torch.empty(0)
  for batch in train_dataloader: # We apply the mini batch here 
    x_train,y_train = batch      # Instanciate x and y regarding the batch (64)
    y_pred = model2.forward(x_train.float())    
    # Our loss function that use the hinge loss and the norm of the weights 
    loss = ((0.5/gamma) * torch.norm(model2.linear.weight.squeeze())**2 )+ (C* torch.clamp(1-y_pred*y_train,min=0)).mean()
    loss.backward()
    total_loss += float(loss)  # We add the loss of this batch to the total loss
    nb_of_batch += 1
    

    # Just to retrieve values of y_train because the dataset is shuffleld and predictions
    y_train_values = torch.cat((y_train_values,y_train),0)
    output = torch.cat((output,y_pred),0)

    with torch.no_grad():  # Here we apply de gradient descned regarding the learning rate and the weights
      for param in model2.parameters():
        param -= learning_rate*param.grad
    model2.zero_grad()
  final_pred = torch.where(output >=0, 2, 3 )
  y_true = torch.where(y_train_values>= 0,2,3)
  print('Epochs : '+ str(epoch) + ' Loss = '+ str(round((total_loss/nb_of_batch),4)) + ' Accuracy = ' + str(accuracy(y_true,final_pred)))

# Final prediction we make the decision if we consider predictions are classe 2 or 3 if their higher than 0

Epochs : 0 Loss = 12.659 Accuracy = 95.59
Epochs : 1 Loss = 10.6457 Accuracy = 96.83
Epochs : 2 Loss = 9.7656 Accuracy = 97.06
Epochs : 3 Loss = 9.2393 Accuracy = 97.17
Epochs : 4 Loss = 8.87 Accuracy = 97.31
Epochs : 5 Loss = 8.5967 Accuracy = 97.39
Epochs : 6 Loss = 8.3963 Accuracy = 97.45
Epochs : 7 Loss = 8.2243 Accuracy = 97.4
Epochs : 8 Loss = 8.0869 Accuracy = 97.5
Epochs : 9 Loss = 7.9576 Accuracy = 97.54


Loss : 123.09158504710479 Accuracy = 97.99


In [84]:
nb_of_batch = 0
loss = 0 
total_loss=0
C= 100
gamma = 10
model2.eval()
output = torch.empty(0)

for batch in val_dataloader: 
    x_val,y_val = batch
    y_pred = model2.forward(x_val)
    loss += ((0.5/gamma) * torch.norm(model2.linear.weight.squeeze())**2 )+ (C* torch.clamp(1-y_pred*y_val,min=0)).mean()
    total_loss += float(loss)
    nb_of_batch += 1
    output = torch.cat((output,y_pred),0)
final_pred = torch.where(output >=0, 2, 3 )

print("Loss : " + str(total_loss/nb_of_batch) + " Accuracy = " + str(float(accuracy(final_pred,y_val_torch_23))))

Loss : 134.17425298690796 Accuracy = 97.61


In [85]:
nb_of_batch = 0
loss = 0 
total_loss=0
gamma = 10
C = 100
model2.eval() # Model in evaluation mode
output = torch.empty(0)
for batch in test_dataloader: 
    x_test,y_test = batch
    y_pred = model2.forward(x_test)
    loss += ((0.5/gamma) * torch.norm(model2.linear.weight.squeeze())**2 )+ (C* torch.clamp(1-y_pred*y_test,min=0)).mean()
    total_loss += float(loss)
    nb_of_batch += 1
    output = torch.cat((output,y_pred),0)
final_pred = torch.where(output >=0, 2, 3 )

print("Loss : " + str(total_loss/nb_of_batch) + " Accuracy = " + str(float(accuracy(final_pred,y_test_torch_23))))

Loss : 128.60798013911528 Accuracy = 97.99


So after trying the best parameters for our model we found the accuracy of 98% for the test set

# Resume

What we have done in this Binary SVM Classification using soft max margin

- Created the SVM classifier
- Trained the model with simple parameters and 10 epochs
- evaluated the model on the validation set
- performed the model on the test set
- Tuned hyperparametrs C and Gamma 
- tested on validation and tests set
- Tuned the learning rate 
- Tested on validation and tests set

## Final accuracy of the test set is 98% with C = 100 gamma = 10 lr= 0.002 

The value of C is very high and the loss will be high to because we penalize more with high value of C our model when it' getting wrong. The value of gamma is pretty because we tried to only consider points that are very close like a unique class